In [441]:
from xspec import *
m = Model("tbabs*bbody+bbodyrad")
s = Spectrum("rxj1856_1k/spectrum_rxj1856_1k.pha")


Model TBabs<1>*bbody<2> + bbodyrad<3> Source No.: 1   Active/On
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   bbody      kT         keV      3.00000      +/-  0.0          
   3    2   bbody      norm                1.00000      +/-  0.0          
   4    3   bbodyrad   kT         keV      3.00000      +/-  0.0          
   5    3   bbodyrad   norm                1.00000      +/-  0.0          
________________________________________________________________________


Fit statistic  : Chi-Squared              9.820652e+07     using 1024 bins, spectrum 1.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1
                 Chi-Squared              9.820652e+07     using 1024 bins, spectrum 2.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 2
                 Chi-Squared         

In [1]:
from xspec import *
# m = XspecModel(exprString="tbabs*bbody", setPars=[1,2,3])
m = Model(exprString="tbabs*bbody", setPars=[1,2,3])
m.expression

tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.


'TBabs*bbody'


Model TBabs<1>*bbody<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   bbody      kT         keV      2.00000      +/-  0.0          
   3    2   bbody      norm                3.00000      +/-  0.0          
________________________________________________________________________



In [27]:
from XspecModel import XspecModel
from xspec import *
m = XspecModel(exprString="tbabs*bbody", setPars=[1, 2, 3]).Model



Model TBabs<1>*bbody<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   bbody      kT         keV      2.00000      +/-  0.0          
   3    2   bbody      norm                3.00000      +/-  0.0          
________________________________________________________________________



In [3]:
AllModels.show()


Parameters defined:
Model TBabs<1>*bbody<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   bbody      kT         keV      2.00000      +/-  0.0          
   3    2   bbody      norm                3.00000      +/-  0.0          
________________________________________________________________________



In [2]:
save_model(m)

In [4]:
m.save()

In [19]:
from xspec import *
from XspecModel import *
mymod = XspecModel("bbodyrad").Model


Model bbodyrad<1> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   bbodyrad   kT         keV      3.00000      +/-  0.0          
   2    1   bbodyrad   norm                1.00000      +/-  0.0          
________________________________________________________________________



In [20]:
save_model(mymod)

In [30]:
# AllModels.clear()
# load_model("Model_bbodyrad.xcm")
Plot.device = "/xs"
Plot("model")

In [31]:
AllModels.show()


Parameters defined:
Model bbodyrad<1> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   bbodyrad   kT         keV      3.00000      +/-  0.0          
   2    1   bbodyrad   norm                1.00000      +/-  0.0          
________________________________________________________________________



In [23]:
Xset.abund = 'wilm'
m = Model("tbabs*bbodyrad")

 Solar Abundance Vector set to wilm:  Wilms, J., Allen, A. & McCray, R. ApJ 542 914 (2000) (abundances are set to zero for those elements not included in the paper).


tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.

Model TBabs<1>*bbodyrad<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   bbodyrad   kT         keV      3.00000      +/-  0.0          
   3    2   bbodyrad   norm                1.00000      +/-  0.0          
________________________________________________________________________



In [24]:
m = load_model("Model_bbodyrad.xcm")

 Solar Abundance Vector set to wilm:  Wilms, J., Allen, A. & McCray, R. ApJ 542 914 (2000) (abundances are set to zero for those elements not included in the paper).
 Cross Section Table set to vern:  Verner, Ferland, Korista, and Yakovlev 1996
Parameter delta values for fits will now be 0.01 * parValue

 The default systematic error for models is set to 0


Model bbodyrad<1> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   bbodyrad   kT         keV      3.00000      +/-  0.0          
   2    1   bbodyrad   norm                1.00000      +/-  0.0          
________________________________________________________________________

